In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
import numpy as np
import pandas as pd

# Download the IMDb movie review dataset
from tensorflow.keras.datasets import imdb
(train_data, train_labels), (test_data, test_labels) = imdb.load_data()

# Combine train and test data for preprocessing
data = np.concatenate((train_data, test_data), axis=0)
labels = np.concatenate((train_labels, test_labels), axis=0)

# Set vocabulary size and maximum sequence length
vocab_size = 10000
max_len = 256

# Preprocess the text data
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts([' '.join(map(str, sentence)) for sentence in data])
sequences = tokenizer.texts_to_sequences([' '.join(map(str, sentence)) for sentence in data])
word_index = tokenizer.word_index
data = pad_sequences(sequences, maxlen=max_len, truncating='post', padding='post')

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


# Define the Transformer model
class TransformerModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, max_len, num_heads, ff_dim, num_blocks, dropout_rate):
        super(TransformerModel, self).__init__()
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.pos_embedding = tf.keras.layers.Embedding(input_dim=max_len, output_dim=embedding_dim)
        self.encoder_blocks = [TransformerEncoderBlock(embedding_dim, num_heads, ff_dim, dropout_rate) for _ in range(num_blocks)]
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(1, activation='sigmoid')

    def call(self, inputs):
        x = self.embedding(inputs)
        positions = tf.range(start=0, limit=tf.shape(x)[-2], delta=1)
        positions = self.pos_embedding(positions)
        x += positions
        for encoder_block in self.encoder_blocks:
            x = encoder_block(x)
        x = self.flatten(x)
        x = self.dense(x)
        return x

class TransformerEncoderBlock(tf.keras.layers.Layer):
    def __init__(self, embedding_dim, num_heads, ff_dim, dropout_rate):
        super(TransformerEncoderBlock, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embedding_dim)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(ff_dim, activation='relu'),
            tf.keras.layers.Dense(embedding_dim)
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization()
        self.layernorm2 = tf.keras.layers.LayerNormalization()
        self.dropout1 = tf.keras.layers.Dropout(dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(dropout_rate)

    def call(self, inputs):
        x = self.layernorm1(inputs)
        attention_output = self.att(x, x)
        attention_output = self.dropout1(attention_output)
        x2 = tf.keras.layers.Add()([inputs, attention_output])
        x = self.layernorm2(x2)
        ffn_output = self.ffn(x)
        ffn_output = self.dropout2(ffn_output)
        x3 = tf.keras.layers.Add()([x2, ffn_output])
        return x3

# Define hyperparameters
embedding_dim = 128
num_heads = 8
ff_dim = 256
num_blocks = 6
dropout_rate = 0.1
batch_size = 32
epochs = 10

# Create an instance of the Transformer model
model = TransformerModel(vocab_size, embedding_dim, max_len, num_heads, ff_dim, num_blocks, dropout_rate)

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Convert the labels to numpy arrays
y_train = np.array(y_train)
y_test = np.array(y_test)

# Train the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


    8192/17464789 [..............................] - ETA: 0s

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


17464789/17464789 [==============================] - 0s 0us/step
Epoch 1/10
1250/1250 [==============================] - 328s 244ms/step - loss: 0.7221 - accuracy: 0.7788 - val_loss: 0.4199 - val_accuracy: 0.8525
Epoch 2/10
1250/1250 [==============================] - 276s 221ms/step - loss: 0.3256 - accuracy: 0.8780 - val_loss: 0.3131 - val_accuracy: 0.8700
Epoch 3/10
1250/1250 [==============================] - 274s 219ms/step - loss: 0.2593 - accuracy: 0.9026 - val_loss: 0.3739 - val_accuracy: 0.8713
Epoch 4/10
1250/1250 [==============================] - 254s 203ms/step - loss: 0.2186 - accuracy: 0.9202 - val_loss: 0.4910 - val_accuracy: 0.8686
Epoch 5/10
1250/1250 [==============================] - 270s 216ms/step - loss: 0.1982 - accuracy: 0.9317 - val_loss: 0.4339 - val_accuracy: 0.8690
Epoch 6/10
1250/1250 [==============================] - 250s 200ms/step - loss: 1.6968 - accuracy: 0.9307 - val_loss: 0.9814 - val_accuracy: 0.8618
Epoch 7/10
1250/1250 [=========================